<a href="https://colab.research.google.com/github/Ian-Ibrahim/SU-Machine-learning-1222785/blob/main/Text_classification122785_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 122785 Lab three

In [ ]:
!pip install datasets -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("pacovaldez/stackoverflow-questions", split = 'validation')
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/pacovaldez___parquet/pacovaldez--stackoverflow-questions-77c9a43a92f08f84/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Dataset({
    features: ['title', 'body', 'label'],
    num_rows: 785098
})

In [ ]:
datasetTexts = dataset['title']
print(len(datasetTexts))

785098


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
sample = datasetTexts[5]
sample_encoding = model.encode(sample)

In [ ]:
embeddings = model.encode(datasetTexts[:700])

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters = 5, random_state = 42)
km.fit(embeddings)

cluster_labels = km.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cs

def search(query, texts, embeddings):
    q_emb = model.encode([query])[0]
    sim_scores = cs([q_emb], embeddings)[0]
    results = [(texts[i], sim_scores[i]) for i in range(len(texts))]
    results.sort(key = lambda x: x[1], reverse = True)
    return results

In [ ]:
query = 'Module not found'
search_results = search(query, datasetTexts[:700], embeddings)

In [ ]:
from tqdm.auto import tqdm

scores = []
titles = []

for i in range(len(tqdm(search_results))):
    scores.append(search_results[i][1])
    titles.append(search_results[i][0])

  0%|          | 0/700 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "Titles": titles,
        "Sim": scores
    }
)

df = df.sort_values('Sim', ascending = False)
df.head(10)

,Titles,Sim
0,ImportError: No module named 'parse',0.663350
1,Module build failed (from ./node_modules/babel...,0.549500
2,eslint + typescript + path in tsconfig: ESLint...,0.467615
3,How do I include a Perl module that's in a dif...,0.453819
4,Error whilst installing magento extension- pac...,0.438409
5,Entity Framework Cast not supported,0.410880
6,ASP.NET framework MVC5 empty project NuGet Pac...,0.408930
7,Failing to compile vlc for android,0.388735
8,Problems While Trying To Compile MacVim,0.387978
9,Eclipse Java Missing required source folder: '...,0.369784


In [ ]:
def deploy(error_text):
    search_results = search(error_text, datasetTexts[:700], embeddings)
    scores = []
    titles = []

    for i in range(len(tqdm(search_results))):
        scores.append(search_results[i][1])
        titles.append(search_results[i][0])

    df = pd.DataFrame({"Titles": titles,"Sim": scores})
    df = df.sort_values('Sim', ascending = False)
    print(df.head(10))

In [ ]:
x = input("input your  error description.....")
deploy(str(x))

input your  error description.....import module: no module named 'pdfkit'


  0%|          | 0/700 [00:00<?, ?it/s]

                                              Titles       Sim
0               ImportError: No module named 'parse'  0.608030
1  ASP.NET framework MVC5 empty project NuGet Pac...  0.480198
2  eslint + typescript + path in tsconfig: ESLint...  0.449636
3  Error whilst installing magento extension- pac...  0.429880
4                 Failing to compile vlc for android  0.408183
5  Module build failed (from ./node_modules/babel...  0.403439
6  How do I include a Perl module that's in a dif...  0.394888
7  Cannot see some PDF annotations in Adobe Reade...  0.371923
8  Permission denied when invoking Document AI v1...  0.366074
9            Problems While Trying To Compile MacVim  0.363147
